nro id : se não tiver length 16 adicionar 0 na frente

instituição
- hospital : não é de ensino, tem q marcar q não é de academia
- mesma instituição, registros e tempos diferentes  =>  tem q agrupar
similaridade de string -> função cosseno de PLN (ngramas)



1) pegar o nome de todas as instituições do brasil (usar a lista do MEC)


- fazer a tabela de frequência /
eliminar oq tem pouca frequência
- se for secretaria municipal algo assim e aparecer poucas vezes => só rotular como servidor público
- se estiver so EACH (sem ter USP no nome), categorizar como 'outros'
- lista de universidades
    - tabela do ROR
    - tabela do MEC

In [4]:
from google.colab import drive
import os

import pandas as pd
import numpy as np

drive.mount('/content/gdrive')
os.chdir('/content/gdrive/My Drive/USP/2024_2º/MQA/')

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

Mounted at /content/gdrive


## READ_CSV

In [21]:
df = pd.read_csv('atuacoes_2.csv', encoding='UTF-8', nrows=10000) #, dtype=str, nrows=1000)

df['ANO-FIM'] = df['ANO-FIM'].fillna(2024)
df['FLAG-DEDICACAO-EXCLUSIVA'] = df['FLAG-DEDICACAO-EXCLUSIVA'].replace('NAO', False).replace('SIM', True)

df = df.astype({
    'NRO-ID-CNPQ': int,
    'CODIGO-INSTITUICAO': str,
    'NOME-INSTITUICAO': str,
    'ANO-INICIO': int,
    'ANO-FIM': int,
    'ENQUADRAMENTO-FUNCIONAL': str,
    'TIPO-DE-VINCULO': str,
    'OUTRAS-INFORMACOES': str,
    'CARGA-HORARIA-SEMANAL': float,
    'FLAG-DEDICACAO-EXCLUSIVA': bool
})

df = df.fillna(' ')
df2 = df.copy()

df

,NRO-ID-CNPQ,CODIGO-INSTITUICAO,NOME-INSTITUICAO,ANO-INICIO,ANO-FIM,ENQUADRAMENTO-FUNCIONAL,TIPO-DE-VINCULO,OUTRAS-INFORMACOES,CARGA-HORARIA-SEMANAL,FLAG-DEDICACAO-EXCLUSIVA
0,3397219421632200,000100000991,HOSPITAL UNIVERSITÁRIO PROFESSOR ALBERTO ANTUNES,2011,2015,LIVRE,LIVRE,Analista Júnior III,40.0,False
1,3397219421632200,000100000991,HOSPITAL UNIVERSITÁRIO PROFESSOR ALBERTO ANTUNES,2016,2017,LIVRE,LIVRE,Secretário Executivo,30.0,False
2,3397219421632200,000200000993,Secretaria de Estado da Educação de Alagoas,2014,2017,LIVRE,LIVRE,Professor Monitor da Disciplina de Geografia,10.0,False
3,3397219421632200,000300000995,Procuradoria Geral do Município de Maceió,2017,2024,SERVIDOR_PUBLICO,LIVRE,Assistente Administrativo,30.0,False
4,3381149537136202,192900000005,Universidade do Estado do Pará,2009,2024,LIVRE,LIVRE,Monitoria,15.0,True
...,...,...,...,...,...,...,...,...,...,...
9995,3310682537361721,008000000006,Universidade Estadual de Londrina,2016,2018,LIVRE,LIVRE,Estudante de Pós-Graduação,20.0,True
9996,3310682537361721,008000000006,Universidade Estadual de Londrina,2020,2024,LIVRE,LIVRE,Estudante de Pós-Graduação,,False
9997,3331949290153820,000500000000,Universidade Federal Fluminense,2006,2006,LIVRE,LIVRE,Estagiaria em Nutrição Clínica,30.0,False
9998,3331949290153820,000500000000,Universidade Federal Fluminense,2006,2006,LIVRE,LIVRE,Estagiaria em Nutrição em Saúde Pública,45.0,False


## TRATAMENTO

In [22]:
# frequency distribution table of df
df2['NOME-INSTITUICAO'].value_counts()

,count
NOME-INSTITUICAO,
Universidade de São Paulo,109
Universidade Federal de Minas Gerais,62
Universidade Estadual Paulista Júlio de Mesquita Filho,57
Universidade Federal do Rio de Janeiro,56
Universidade Federal de Pernambuco,53
...,...
Sindicato do Serviço Público Federal no Estado do Pará,1
Centro de Formação para a Educação Infantil,1
FACULDADE DE EDUCAÇÃO SUPERIOR DO PIEMONTE DA CHAPADA,1


In [23]:
# drop frequency less than 100
df2 = df2[df2.groupby('NOME-INSTITUICAO')['NOME-INSTITUICAO'].transform('count').ge(50)]
df2['NOME-INSTITUICAO'].value_counts()

In [31]:
# LISTA DE UNIVERSIDADES
# https://dadosabertos.mec.gov.br/indicadores-sobre-ensino-superior/item/181-instituicoes-de-educacao-superior-do-brasil
# https://dadosabertos.mec.gov.br/images/conteudo/Ind-ensino-superior/2022/PDA_Lista_Instituicoes_Ensino_Superior_do_Brasil_EMEC.csv

# criar uma categoria '0' : educacao

df_uni = pd.read_csv('sup.csv', encoding='UTF-8')
df_uni_filtered = df_uni[['NOME_DA_IES', 'SIGLA']]
df_uni_filtered = df_uni_filtered.fillna('')
df_uni_filtered = df_uni_filtered.apply(lambda x: x.astype(str).str.upper())

df_uni_filtered

,NOME_DA_IES,SIGLA
0,FACULDADE DE TECNOLOGIA SENAC CURITIBA,
1,FACULDADE REGIONAL SERRANA,FUNPAC
2,UNIVERSIDADE DO ESTADO DO AMAPÁ,UEAP
3,FACULDADE SÃO JUDAS DE SÃO BERNARDO DO CAMPO,
4,FACULDADE DE CIÊNCIAS GERENCIAIS ALVES FORTES ...,FACE ALFOR JF
...,...,...
4323,FACULDADE DE NEGÓCIOS,FTEC NEGÓCIOS
4324,FACULDADE DE RONDÔNIA,FARO
4325,FACULDADE DE SAÚDE TECSOMA,FASTEC
4326,FACULDADE DE CIÊNCIAS GERÊNCIAIS DE VILHENA,FCGV


In [ ]:
# merge ANO-INICIO and ANO-FIM with same CODIGO-INSTITUICAO and NRO-ID-CNPQ

df_merged = df.groupby(['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO']).agg(
    earliest_date1=('ANO-INICIO', 'min'),
    latest_date2=('ANO-FIM', 'max')
).reset_index()


# df.loc[df['NRO-ID-CNPQ'] == 3300778291054405]

In [ ]:
# merge column NOME-INSTITUICAO with df_merged

df_drop_duplicates = df.drop_duplicates(subset=['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO'])
df_inst_merged = pd.merge(df_merged, df_drop_duplicates, on=['CODIGO-INSTITUICAO', 'NRO-ID-CNPQ'], how='left')
df_inst_merged['NOME-INSTITUICAO'] = df_inst_merged['NOME-INSTITUICAO'].str.upper()

df_inst_merged = df_inst_merged[['NRO-ID-CNPQ', 'CODIGO-INSTITUICAO', 'earliest_date1', 'latest_date2', 'NOME-INSTITUICAO']]
df_inst_merged

,NRO-ID-CNPQ,CODIGO-INSTITUICAO,earliest_date1,latest_date2,NOME-INSTITUICAO
0,3300004683185243,000100000991,2015,2018,MENEZES NIEBUHR ADVOGADOS ASSOCIADOS
1,3300004683185243,000500000999,2011,2013,TRIBUNAL DE JUSTIÇA DE SANTA CATARINA
2,3300004683185243,000700000992,2020,2024,REVISTA DE DEFESA DA CONCORRÊNCIA - RDC
3,3300004683185243,J16T00000000,2018,2024,CONSELHO ADMINISTRATIVO DE DEFESA ECONÔMICA
4,3300006317745526,000100000991,2008,2024,FACULDADE SANTO AGOSTINHO
...,...,...,...,...,...
5196512,6699932849737215,000200000993,2008,2009,FACULDADE UNIDA DA PARAIBA
5196513,6699932849737215,000300000995,2014,2024,FACULDADE DE MEDICINA NOVA ESPERANÇA
5196514,6699932849737215,008300000001,2001,2008,UNIVERSIDADE FEDERAL DA PARAÍBA
5196515,6699932849737215,IXGI00000007,2011,2024,FACULDADE SANTA MARIA DE CAJAZEIRAS


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# The target word for filtering (single word)
target_word = "UNIVERSIDADE DE SÃO PAULO"

# Step 1: Turn phrases into TF-IDF vectors
vectorizer = TfidfVectorizer()
phrase_vectors = vectorizer.fit_transform(df_inst_merged['NOME-INSTITUICAO'])

# Step 2: Convert the target word into a vector
# We will use the same vectorizer to transform the target word
target_vector = vectorizer.transform([target_word])

# Step 3: Compute cosine similarity between the target word and all phrases
similarity_scores = cosine_similarity(target_vector, phrase_vectors).flatten()

df_inst_merged['PREDICTION'] = similarity_scores

# Step 4: Filter phrases based on a similarity threshold (e.g., > 0.1)
threshold = 0.7
#filtered_phrases = df_inst_merged[similarity_scores > threshold]

# Display the phrases that have similarity above the threshold
#print("Filtered Phrases based on Cosine Similarity with the word 'sky':")
#print(filtered_phrases)
df_inst_merged

,NRO-ID-CNPQ,CODIGO-INSTITUICAO,earliest_date1,latest_date2,NOME-INSTITUICAO,PREDICTION
0,3300004683185243,000100000991,2015,2018,MENEZES NIEBUHR ADVOGADOS ASSOCIADOS,0.000000
1,3300004683185243,000500000999,2011,2013,TRIBUNAL DE JUSTIÇA DE SANTA CATARINA,0.075836
2,3300004683185243,000700000992,2020,2024,REVISTA DE DEFESA DA CONCORRÊNCIA - RDC,0.021339
3,3300004683185243,J16T00000000,2018,2024,CONSELHO ADMINISTRATIVO DE DEFESA ECONÔMICA,0.029111
4,3300006317745526,000100000991,2008,2024,FACULDADE SANTO AGOSTINHO,0.000000
...,...,...,...,...,...,...
5196512,6699932849737215,000200000993,2008,2009,FACULDADE UNIDA DA PARAIBA,0.000000
5196513,6699932849737215,000300000995,2014,2024,FACULDADE DE MEDICINA NOVA ESPERANÇA,0.033797
5196514,6699932849737215,008300000001,2001,2008,UNIVERSIDADE FEDERAL DA PARAÍBA,0.131799
5196515,6699932849737215,IXGI00000007,2011,2024,FACULDADE SANTA MARIA DE CAJAZEIRAS,0.034507


In [ ]:
df_inst_merged.to_csv("atuacoes_2_filtrado.csv", encoding='utf8')